# Ejercicio 01

## Objetivo
Crear un análisis integrado de campañas publicitarias combinando datos de Google Ads, Facebook Ads y Google Analytics, realizando un análisis predictivo simple y visualizando los resultados.


In [ ]:
from facebook_business.adobjects.adaccount import AdAccount
from gaarf.report_fetcher import AdsReportFetcher
from facebook_business.api import FacebookAdsApi
from datetime import datetime, timedelta
from gaarf import GoogleAdsApiClient
from collections import defaultdict
from dotenv import load_dotenv
import plotly.express as px
from auth import ga_auth
import statsmodels
import polars as pl
import os

# Cargar variables de entorno
load_dotenv()

## 1. Configuración de Variables
Definir las fechas y cuentas para el análisis:

In [ ]:
# Tu código aquí:
# 1. Define una fecha fin (end_date) usando datetime.now()
# 2. Define una fecha inicio (start_date) 30 días antes que end_date
# 3. Define los IDs de las cuentas de Google Ads, Facebook Ads y Analytics


# Resultados esperados similares a:
# end_date = "2024-01-15"
# start_date = "2023-12-15"
# google_ads_account = "1234567890"
# facebook_ads_account = "9876543210"
# analytics_property = "1112223330"

## 2. Extracción de Datos de Google Ads
Obtén los datos de Google Ads usando la librería gaarf:

In [ ]:
# Tu código aquí:
# 1. Crea el cliente de Google Ads
# 2. Crea una query GAQL para obtener:
#    - date
#    - campaign name
#    - cost
#    - impressions
#    - clicks
# 3. Ejecuta la query y convierte a DataFrame de Polars
# 4. Renombra las columnas y ajusta los tipos de datos según sea necesario

# PISTA: Para el costo, recuerda dividir cost_micros entre 1,000,000


## 3. Extracción de Datos de Facebook Ads
Obtén los datos de Facebook Ads usando el SDK oficial:

In [ ]:
# Tu código aquí:
# 1. Inicializa la API de Facebook
# 2. Define los campos requeridos (similares a Google Ads)
# 3. Define los parámetros de la consulta (fechas y nivel de agregación)
# 4. Obtén los insights y conviértelos a DataFrame de Polars
# 5. Asegúrate que los nombres de columnas coincidan con el DataFrame de Google Ads

In [ ]:
df_facebook_ads = df.with_columns([
    (pl.lit("facebook").alias("medio")),
    (pl.col("date_start").str.to_date("%Y-%m-%d").alias("date")),
    (pl.col("campaign_name").alias("campaign")),
    (pl.col("spend").cast(pl.Float64).alias("cost")),
    (pl.col("impressions").cast(pl.Int64)),
    (pl.col("clicks").cast(pl.Int64))
]).select([
    "date",
    "medio",
    "campaign",
    "cost",
    "impressions",
    "clicks"
    ])

## 4. Extracción de Datos de Google Analytics
Para obtener datos de Analytics, usaremos una función helper proporcionada:

In [ ]:
def ga_data(property_id, start_date, end_date, dimensions, metrics, filter_expr=None):
    analytics = ga_auth(['https://www.googleapis.com/auth/analytics.readonly'])
    request = {
        "requests": [{
            "dateRanges": [{
                "startDate": start_date,
                "endDate": end_date
            }],
            "dimensions": [{"name": dim} for dim in dimensions],
            "metrics": [{"name": met} for met in metrics],
            "limit": 250000
        }]
    }
    
    # Agregar filtro si se especifica
    if filter_expr:
        request["requests"][0]["dimensionFilter"] = {
            "filter": filter_expr
        }
    
    # Obtener datos
    response = analytics.properties().batchRunReports(
        property=f'properties/{property_id}', 
        body=request
    ).execute()
    
    # Procesar respuesta
    data = defaultdict(list)
    for row in response['reports'][0]['rows']:
        # Agregar dimensiones
        for dim, value in zip(dimensions, row['dimensionValues']):
            data[dim].append(value['value'])
        # Agregar métricas
        for met, value in zip(metrics, row['metricValues']):
            data[met].append(value['value'])
            
    return pl.DataFrame(data)


In [ ]:
# Tu código aquí:
# 1. Define las dimensiones necesarias (date, sessionCampaignName)
# 2. Define las métricas necesarias (sessions, transactions, purchaseRevenue)
# 3. Crea un filtro para campañas (opcional)
# 4. Ejecuta la función y procesa el resultado

## 5. Integración de Datos
Combina los datos de las tres fuentes:

In [ ]:
# Tu código aquí:
# 1. Combina los DataFrames de Google Ads y Facebook Ads usando pl.concat
# 2. Realiza un left join con los datos de Analytics
# 3. Verifica que las columnas estén correctamente alineadas

## 6. Análisis y Visualización
Crea visualizaciones para entender los datos:

In [ ]:
# Tu código aquí:
# 1. Crea un gráfico de barras comparando costos por plataforma
# 2. Crea un gráfico de barras comparando ingresos por plataforma
# 3. Visualiza la tendencia de costos en el tiempo
# 4. Visualiza la tendencia de ingresos en el tiempo
# 5. Crea un scatter plot de costo vs ingresos con línea de tendencia
# 6. Bonus: Crea scatter plots separados por plataforma

## Recursos Adicionales:
- Documentación de Polars: https://pola-rs.github.io/polars/
- Documentación de Plotly Express: https://plotly.com/python-api-reference/plotly.express.html
- Guía de GAQL: https://developers.google.com/google-ads/api/docs/query/overview